# AI Toolkit by Ostris
## Memecoin Multi-Concept LoRA Training


In [ ]:
!nvidia-smi


In [ ]:
import os
import subprocess

if os.path.exists('doom-ai-toolkit'):
    subprocess.run(['git', '-C', 'doom-ai-toolkit', 'pull', 'origin', 'main'], check=True)
    print('Repository updated successfully')
else:
    subprocess.run(['git', 'clone', 'https://github.com/posaune0423/doom-ai-toolkit.git'], check=True)
    print('Repository cloned successfully')

Put your image dataset in the `dataset/` folder at the repo root. This repo already includes the memecoin data in that location, so no extra setup is required.


In [ ]:
import subprocess
import sys
import os

# Install jedi first (required by ipython)
print("Installing jedi...")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "jedi>=0.16"], check=True)

# Install numpy compatible with easy-dwpose (numpy<2.0.0 required)
# Note: Colab environment may have packages requiring numpy>=2.0, but they are not part of this project
print("Installing numpy...")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "numpy>=1.26.4,<2.0.0"], check=True)

# Reinstall PyTorch to ensure binary compatibility with the numpy version we just installed
# This is critical to avoid "numpy.dtype size changed" errors
print("Checking PyTorch installation...")
result = subprocess.run([sys.executable, "-m", "pip", "show", "torch"], capture_output=True, text=True)
if result.returncode == 0:
    print("PyTorch is already installed, reinstalling for numpy compatibility...")
    subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall",
                    "torch", "torchvision", "torchaudio"], check=True)
else:
    print("PyTorch not found, installing...")
    subprocess.run([sys.executable, "-m", "pip", "install",
                    "torch", "torchvision", "torchaudio"], check=True)

# Change to doom-ai-toolkit directory and update submodules
print("Updating git submodules...")
os.chdir('doom-ai-toolkit')
subprocess.run(["git", "submodule", "update", "--init", "--recursive"], check=True)

# Install huggingface_hub with correct version first (before transformers)
# This fixes the ImportError: huggingface-hub>=0.30.0,<1.0 is required
print("Installing huggingface_hub with correct version...")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall",
                "huggingface_hub>=0.30.0,<1.0"], check=True)

# Install dependencies from requirements.txt (but skip diffusers and easy-dwpose for now)
print("Installing dependencies from requirements.txt...")
# Read requirements.txt and filter out diffusers and easy-dwpose
with open('requirements.txt', 'r') as f:
    requirements = f.readlines()

filtered_requirements = []
for req in requirements:
    req = req.strip()
    if req and not req.startswith('#') and 'diffusers' not in req and 'easy_dwpose' not in req:
        filtered_requirements.append(req)

# Install filtered requirements
if filtered_requirements:
    subprocess.run([sys.executable, "-m", "pip", "install"] + filtered_requirements, check=True)

# Reinstall diffusers to ensure binary compatibility with numpy
# This fixes the "numpy.dtype size changed" error when importing diffusers.schedulers
# See: https://github.com/ostris/ai-toolkit/issues/267
print("Reinstalling diffusers for numpy compatibility...")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall",
                "git+https://github.com/huggingface/diffusers@1448b035859dd57bbb565239dcdd79a025a85422"], check=True)

# Install easy-dwpose separately with --no-deps to avoid numpy version conflicts
# easy-dwpose requires numpy<2.0.0, which we've already installed above
print("Installing easy-dwpose...")
subprocess.run([sys.executable, "-m", "pip", "install", "--no-deps",
                "git+https://github.com/jaretburkett/easy_dwpose.git"], check=True)

print("All dependencies installed successfully!")

## Model License
FLUX.1-schnell is Apache 2.0 and does not require a gated Hugging Face token, but you still need to agree to the model terms on Hugging Face and keep a token handy if you plan to access private models. Use the next cell to set `HF_TOKEN` when needed.


In [ ]:
import os

SECRET_NAME = "HF_TOKEN"

from google.colab import userdata  # type: ignore

hf_token = userdata.get(SECRET_NAME)
if not hf_token:
    raise RuntimeError(
        f"Failed to retrieve required Hugging Face token from Colab secret '{SECRET_NAME}'."
    )

os.environ["HF_TOKEN"] = hf_token
print(f"HF_TOKEN environment variable has been set from Colab secret '{SECRET_NAME}'.")


In [ ]:
import os
import sys
sys.path.append('/content/doom-ai-toolkit')
from toolkit.job import run_job
import yaml
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"


## Setup

This is your config. It is documented pretty well. The configuration is loaded from a YAML file. This will run as is without modification, but feel free to edit as you want.


In [ ]:
from pathlib import Path

config_path = Path('/content/doom-ai-toolkit/config/train_memecoin_multi_lora.yaml')
job_config = yaml.safe_load(config_path.read_text())
print('Configuration loaded')
print(f"Training: {job_config['config']['name']}")
print(f"Datasets: {len(job_config['config']['process'][0]['datasets'])}")


## Run it

Below does all the magic. Check your folders to the left. Items will be in output/memecoin_multi_lora_v1. In the samples folder, there are periodic samples. This doesn't work great with colab. They will be in /content/doom-ai-toolkit/output


In [ ]:
run_job(job_config)


## Done

Check your output dir and get your LoRA


In [ ]:
from IPython.display import Image, display
import os

samples_dir = '/content/doom-ai-toolkit/output/memecoin_multi_lora_v1/samples'
if os.path.exists(samples_dir):
    images = sorted([f for f in os.listdir(samples_dir) if f.endswith('.png')])
    for img in images[-6:]:
        display(Image(filename=os.path.join(samples_dir, img)))
